In [103]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from mlserve_sdk.client import MLServeClient
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv

load_dotenv()

True

In [104]:
# -----------------------------
# 1. Connect to MLServe
# -----------------------------
USERNAME = os.getenv("USERNAME")
TOKEN = os.getenv("TOKEN")

client = MLServeClient()
client.login(USERNAME, TOKEN)

In [105]:
# -----------------------------
# 2. Train a sample model
# -----------------------------
data = load_iris(as_frame=True)
X = data['data']
y = data['target']
model = LogisticRegression(max_iter=200)
model.fit(X, y)
score=model.score(X, y)

In [106]:
# -----------------------------
# 3. Deploy model
# -----------------------------
feats=data['feature_names']
client.deploy(
    model=model,
    name="iris",
    version="v10",
    features=feats,
    background_df=X,
    task_type='classification',
    metrics={'accuracy':score}
)

{'predict_url': 'https://mlserve.com/api/v1/predict/iris/v10'}

In [129]:
# -----------------------------
# 4. Predict (specific version)
# -----------------------------
iris_data = {
    "inputs": [
        [5.1, 3.5, 1.4, 0.2],  # Iris-setosa
    ]
}
preds = client.predict("iris", "v10", iris_data, explain=True)
print("Predictions:", preds)

Predictions: {'version': 'v10', 'prediction_ids': ['e4f61b51-2b3f-4ba3-90d1-b09d16045e8a'], 'predictions': [0], 'explanations': [[{'feature': 'petal length (cm)', 'value': 1.4, 'shap_value': 5.623, 'impact': 'positive'}, {'feature': 'petal width (cm)', 'value': 0.2, 'shap_value': 1.021, 'impact': 'positive'}, {'feature': 'sepal width (cm)', 'value': 3.5, 'shap_value': 0.412, 'impact': 'positive'}]]}


In [130]:
# -----------------------------
# 7. Fetch Basic Metrics
# -----------------------------
metrics = client.get_metrics("iris", "v10", as_dataframe=True)
metrics

,requests,predictions,throughput_rps,prediction_rps,avg_latency_ms,p50_latency_ms,p95_latency_ms,p99_latency_ms,avg_latency_per_element_ms,p50_latency_per_element_ms,p95_latency_per_element_ms,p99_latency_per_element_ms,error_rate
timestamp,,,,,,,,,,,,,
2025-09-18 07:00:00,21,21,0.005833,0.005833,26.612702,17.409378,92.187309,131.462617,26.612702,17.409378,92.187309,131.462617,0.0
